# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля.<br/>
<br/>
Нам необходимо построить эффективную модель для прогнозирования рыночной стоимости автомобиля на базе данных о технических характеристиках, комплектации и ценах автомобилей.<br/>
<br/>
Заказчику в модели предсказания важны:
- качество предсказания;
- время обучения модели;
- время предсказания модели.

Исходный датасет содержит 16 столбцов и 354369 строк. <br/>
<br/>
<b>Признаки:</b> <br/>
<br/>
DateCrawled — дата скачивания анкеты из базы<br/>
VehicleType — тип автомобильного кузова<br/>
RegistrationYear — год регистрации автомобиля<br/>
Gearbox — тип коробки передач<br/>
Power — мощность (л. с.)<br/>
Model — модель автомобиля<br/>
Kilometer — пробег (км)<br/>
RegistrationMonth — месяц регистрации автомобиля<br/>
FuelType — тип топлива<br/>
Brand — марка автомобиля<br/>
NotRepaired — была машина в ремонте или нет<br/>
DateCreated — дата создания анкеты<br/>
NumberOfPictures — количество фотографий автомобиля<br/>
PostalCode — почтовый индекс владельца анкеты (пользователя)<br/>
LastSeen — дата последней активности пользователя<br/>
<br/>
    <b>Целевой признак:</b> Price — цена (евро)<br/>
<br/>
В процессе работы над проектом необходим подготовить данные к обучению, исследовать несколько моделей ML и выбрать наиболее оптимальную. 


## Подготовка данных

### Предобработка данных.

In [ ]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import lightgbm as lgb
import catboost as cb

In [ ]:
try:
  df= pd.read_csv('/datasets/autos.csv')
except FileNotFoundError:
  df = pd.read_csv('/content/autos.csv')

df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Проверим данные на дубликаты

In [ ]:
df.duplicated().sum()

4

In [ ]:
df = df.drop_duplicates()
df.duplicated().sum()

0

Проверим категориальные признаки на скрытые дубликаты.

In [ ]:
columns = ['VehicleType','Gearbox','Model','FuelType','Brand','Repaired']
for column in columns:
    print(column)
    print(df[column].unique())
    print()

VehicleType
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']

Gearbox
['manual' 'auto' nan]

Model
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuor

К неявным дубликатам относятся названия модели 'rangerover' и 'range_rover'. Приведем название к единому стилю.

In [ ]:
df['Model'] = df['Model'].replace('range_rover', 'rangerover')
df.query('Model == "range_rover"')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen


Столбцы DateCrawled, DateCreated, LastSeen, PostalCode не несут значимую для обучения моделей информацию, поэтому удалим их.

In [ ]:
df = df.drop(['NumberOfPictures','DateCrawled','DateCreated','LastSeen','PostalCode'],axis=1)
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no


Посмотрим основные статистические характеристики данных по каждому числовому признаку.

In [ ]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,354365.000000,354365.000000,354365.000000,354365.000000,354365.000000
mean,4416.679830,2004.234481,110.093816,128211.363989,5.714650
std,4514.176349,90.228466,189.851330,37905.083858,3.726432
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


- Средняя стоимость автомобиля 2700 евро. Минимальная цена авто - 0 евро. Возможно, автомобиль отдают бесплатно. Оставим эти данные без изменения.
- Минимальный год регистриции автомобиля 1000, максимальный - 9999. Такого быть не может. Далее заполним выбросы единой заглушкой.
- Максимальное значение мощности автомобиля - 20 000 л.с, минимально - 0. Такого тоже быть не может. Далее заполним выбросы медианами.
- Больше половины автомобилей имеют максимальный пробег. Возможно, это ограничение системы.
- Месяц регистрации автомобиля варьируется от 0 до 12. Возможно, 0 это ошибка в данных. 

Посмотрим характеристики по категориальным признакам.

In [ ]:
df.describe(include=['object', 'bool'])

,VehicleType,Gearbox,Model,FuelType,Brand,Repaired
count,316875,334532,334660,321470,354365,283211
unique,8,2,249,7,40,2
top,sedan,manual,golf,petrol,volkswagen,no
freq,91457,268249,29232,216349,77012,247158


- 8 типов автомобильного кузова, чаще всего встречается седан.
- 2 типа коробки передач, чаще встречаются автомобили с ручной коробкой передач.
- 250 уникальных моделей автомобилей. Самый популярный golf. 
- 7 видов топлива, чаще встречаются автомобили на бензине.
- 40 марок автомобилей. Самый популярный бренд Volkswagen.
- большинство автомобилей не были в ремонте, но в столбце очень много пропусков.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354365 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354365 non-null  int64 
 1   VehicleType        316875 non-null  object
 2   RegistrationYear   354365 non-null  int64 
 3   Gearbox            334532 non-null  object
 4   Power              354365 non-null  int64 
 5   Model              334660 non-null  object
 6   Kilometer          354365 non-null  int64 
 7   RegistrationMonth  354365 non-null  int64 
 8   FuelType           321470 non-null  object
 9   Brand              354365 non-null  object
 10  Repaired           283211 non-null  object
dtypes: int64(5), object(6)
memory usage: 32.4+ MB


#### Выводы
Первоначально датафрейм состоял 16 столбцов и 354369 строк. 
В процессе предобработки удалили дубликаты и столбцы, несущественные для последующего обучения. В результате осталось 11 столбцов и 354368 строк. <br/>
В датафрейме довольно много пропусков и нереалистичных данных. Перед обучением необходимо с ними поработать.

### Заполнение пропусков, исправление выбросов.

Перед обучением модели более подробно изучим данные, а также заполним пропуски и исправим выбросы.

In [ ]:
(df.isna().sum() / df.shape[0] * 100).round(2)

Price                 0.00
VehicleType          10.58
RegistrationYear      0.00
Gearbox               5.60
Power                 0.00
Model                 5.56
Kilometer             0.00
RegistrationMonth     0.00
FuelType              9.28
Brand                 0.00
Repaired             20.08
dtype: float64

Самое большое количество пропусков в столбце "Ремонт авто" - 20%. В столбце "Тип кузова" и "Тип топлива" пропусков почти 10%. 

<b>Категориальные признаки</b>

In [ ]:
df['Model'].value_counts().head()

golf     29232
other    24420
3er      19761
polo     13066
corsa    12570
Name: Model, dtype: int64

Other вторая по популярности категория в столбце Модель автомобиля. Заполним пропуски этой же категорией. 

In [ ]:
df['Model'] = df['Model'].fillna('other')

Заполним пропуски в столбцах Коробка передач, тип топлива, ремонт автомобиля значением unknown.

In [ ]:
df['Gearbox'] = df['Gearbox'].fillna('unknown')
df['FuelType'] = df['FuelType'].fillna('unknown')
df['Repaired'] = df['Repaired'].fillna('unknown')

In [ ]:
df['VehicleType'] = df['VehicleType'].fillna('unknown')

In [ ]:
df.isna().sum()

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
dtype: int64

<b>Количественные признаки</b>

In [ ]:
perc = np.arange(0,1,0.03)
df['Power'].describe(percentiles = perc)

count    354365.000000
mean        110.093816
std         189.851330
min           0.000000
0%            0.000000
3%            0.000000
6%            0.000000
9%            0.000000
12%          43.000000
15%          54.000000
18%          60.000000
21%          60.000000
24%          66.000000
27%          73.000000
30%          75.000000
33.0%        75.000000
36%          82.000000
39%          90.000000
42%          94.000000
45.0%       101.000000
48%         102.000000
50%         105.000000
51%         105.000000
54%         109.000000
57.0%       115.000000
60%         116.000000
63%         122.000000
66.0%       125.000000
69%         135.000000
72%         140.000000
75%         143.000000
78%         150.000000
81%         150.000000
84%         163.000000
87%         170.000000
90.0%       179.000000
93%         200.000000
96%         224.000000
99%         300.000000
max       20000.000000
Name: Power, dtype: float64

Как мы видим 99% автомобилей имеют мощность до 300 л.с. Заменим наши верхние и нижние выбросы медианными значениями по модели автомобиля.

In [ ]:
len(df.query('Power == 0 or Power > 300'))

43730

In [ ]:
medians = df.groupby('Model')['Power'].transform('median')
df.loc[df['Power'] == 0, 'Power'] = medians
df.loc[df['Power'] > 300, 'Power'] = medians
df['Power'].describe()

count    354365.000000
mean        116.766226
std          48.652873
min           0.000000
25%          75.000000
50%         107.000000
75%         143.000000
max         343.000000
Name: Power, dtype: float64

In [ ]:
len(df.query('Power == 0'))

10

После заполнения с нулевыми значениями осталось 10 строк. Оставим их без измениния.

<b>RegistrationYear</b>

In [ ]:
perc = np.arange(0,1,0.03)
df['RegistrationYear'].describe(percentiles = perc)

count    354365.000000
mean       2004.234481
std          90.228466
min        1000.000000
0%         1000.000000
3%         1990.000000
6%         1993.000000
9%         1995.000000
12%        1996.000000
15%        1997.000000
18%        1998.000000
21%        1998.000000
24%        1999.000000
27%        1999.000000
30%        2000.000000
33.0%      2000.000000
36%        2001.000000
39%        2001.000000
42%        2002.000000
45.0%      2002.000000
48%        2003.000000
50%        2003.000000
51%        2003.000000
54%        2004.000000
57.0%      2004.000000
60%        2005.000000
63%        2005.000000
66.0%      2006.000000
69%        2006.000000
72%        2007.000000
75%        2008.000000
78%        2008.000000
81%        2009.000000
84%        2010.000000
87%        2011.000000
90.0%      2012.000000
93%        2015.000000
96%        2017.000000
99%        2018.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

Большинство данных по году регистрации автомобиля входят норму. Выбросов не более 3-4%. Чтобы не удалять эти строки, заполним выброс заглужкой -1. Ограничим год регистрации 1930 - 2016

In [ ]:
df.loc[df['RegistrationYear'] < 1930 , 'RegistrationYear'] = -1
df.loc[df['RegistrationYear'] > 2016, 'RegistrationYear'] = -1
df['RegistrationYear'].describe()

count    354365.000000
mean       1919.328808
std         399.734463
min          -1.000000
25%        1998.000000
50%        2002.000000
75%        2007.000000
max        2016.000000
Name: RegistrationYear, dtype: float64

In [ ]:
df.isna().sum()

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
dtype: int64

#### Выводы
В процессе работы с пропусками и выбросами получили следущие результаты:
 - В столбце Модель автомобиля все пропуски заполнили категорией other.
 - Пропуски в столбцах Коробка передач, Тип топлива, Ремонтировался ли автомобиль заполнили значением unknown.
 - Нереалистичные данные в столбце Мощность автомобиля заполнили медианными значениями по моделям автомобиля.
 - Чтобы не терять данные, нереалистичные показатели в столбце Год регистрации автомобиля заполнили заглушкой -1.
Остальные данные сохранили в первоначальном виде.

In [ ]:
df = df.loc[df['Price'] > 0]
df[df['Price'] == 0]

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired


## Обучение моделей

### Подготовка данных к обучению.

Для того, чтобы работать с категориальными данными нам необходимо их преобразовать. Для этого воспользуемся техникой Ordinal Encoder. 

In [ ]:
df_ordinal = df.copy()
category = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
encoder = OrdinalEncoder()
encoder.fit(df_ordinal[category])
df_ordinal[category] = encoder.transform(df_ordinal[category])
df_ordinal

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
0,480,7.0,1993,1.0,95.0,116.0,150000,0,6.0,38.0,1.0
1,18300,2.0,2011,1.0,190.0,166.0,125000,5,2.0,1.0,2.0
2,9800,6.0,2004,0.0,163.0,117.0,125000,8,2.0,14.0,1.0
3,1500,5.0,2001,1.0,75.0,116.0,150000,6,6.0,38.0,0.0
4,3600,5.0,2008,1.0,69.0,101.0,90000,7,2.0,31.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,0.0,2000,1.0,120.0,248.0,150000,3,6.0,24.0,0.0
354365,2200,7.0,2005,2.0,99.0,166.0,20000,1,7.0,33.0,1.0
354366,1199,1.0,2000,0.0,101.0,106.0,125000,3,6.0,32.0,0.0
354367,9200,0.0,1996,1.0,102.0,223.0,150000,3,2.0,38.0,0.0


In [ ]:
df_ohe = pd.get_dummies(df, drop_first=True) # Чтобы избежать дамми-ловушки используем аргумент drop_first
df_ohe

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_unknown,Repaired_yes
0,480,1993,95.0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,18300,2011,190.0,125000,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163.0,125000,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1500,2001,75.0,150000,6,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,3600,2008,69.0,90000,7,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,2000,120.0,150000,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354365,2200,2005,99.0,20000,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
354366,1199,2000,101.0,125000,3,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
354367,9200,1996,102.0,150000,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Разобьем данные на 2 выборки: обучающую и тестовую. Качество обучения будем определять при помощи крос-валидации.

In [ ]:
# для начала разобьем выборку на тестовую и остальную часть
df_train, df_test = train_test_split(df_ordinal, test_size=0.2, random_state=12345) 

# разбиваем теперь оставшуюся выборку на тренировочную и валидационную
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)


# Для линейной регрессии подготовим выборки с использованием датасета после прямого кодирования.

df_train_lg, df_test_lg = train_test_split(df_ohe, test_size=0.2, random_state=12345) 
df_train_lg, df_valid_lg = train_test_split(df_train_lg, test_size=0.25, random_state=12345)

Создаем признаки и целевой признак.

In [ ]:
# признаки для тренировочной выборки
features_train = df_train.drop('Price', axis=1) 
target_train = df_train['Price']

# признаки для валидационной выборки. На ней мы будем проверять качество обучения.
features_valid = df_valid.drop('Price', axis=1)
target_valid = df_valid['Price']

# признаки для тестовой выборки. На ней мы проверим результат обучения.
features_test = df_test.drop('Price', axis=1)
target_test = df_test['Price']


# тоже самое сделаем для линейной регрессии.
features_train_lg = df_train_lg.drop('Price', axis=1)
target_train_lg = df_train_lg['Price']

features_valid_lg = df_valid_lg.drop('Price', axis=1)
target_valid_lg = df_valid_lg['Price']

features_test_lg = df_test_lg.drop('Price', axis=1)
target_test_lg = df_test_lg['Price']

Масштабируем наши количественные признаки.

In [ ]:
# создаем список со столбцами с количественными данными.
numeric = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']
# трансформируем данные
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None;
display(features_train.head())

scaler_lg = StandardScaler()
scaler_lg.fit(features_train_lg[numeric])
features_train_lg[numeric] = scaler_lg.transform(features_train_lg[numeric])
features_valid_lg[numeric] = scaler_lg.transform(features_valid_lg[numeric])
features_test_lg[numeric] = scaler_lg.transform(features_test_lg[numeric])
pd.options.mode.chained_assignment = None;
features_train_lg.head()

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired
86995,0.0,0.205999,1.0,0.249174,212.0,0.577540,1.140897,2.0,20.0,2.0
84015,5.0,0.211071,1.0,-1.085169,101.0,-1.553067,-0.210889,6.0,31.0,0.0
232077,4.0,0.211071,0.0,3.759523,116.0,0.577540,-1.562676,7.0,38.0,0.0
63882,8.0,0.160348,1.0,-0.859358,53.0,0.577540,-1.562676,7.0,9.0,1.0
93078,5.0,0.195854,1.0,-1.167283,8.0,0.577540,1.411254,7.0,25.0,2.0


,RegistrationYear,Power,Kilometer,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_unknown,Repaired_yes
86995,0.205999,0.249174,0.577540,1.140897,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
84015,0.211071,-1.085169,-1.553067,-0.210889,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
232077,0.211071,3.759523,0.577540,-1.562676,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
63882,0.160348,-0.859358,0.577540,-1.562676,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
93078,0.195854,-1.167283,0.577540,1.411254,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


### Подготовка критериев качества модели.

Критерии, которые важны заказчику:
- качество предсказания;
- время обучения модели;
- время предсказания модели.
Для определения качества предсказания будем использовать RMSE. Напишем функцию расчета.

In [ ]:
def RMSE(target, predict):
    return (mean_squared_error(target, predict))**0.5

Для начала попробуем классические модели обучения, а потом перейдем к более сложным, в том числе градиентный бустинг и 

### Линейная регрессию

In [ ]:
%%time

model_linereg = LinearRegression()
model_linereg.fit(features_train_lg, target_train_lg)

CPU times: user 20.9 s, sys: 38.8 s, total: 59.7 s
Wall time: 1min


LinearRegression()

In [ ]:
%%time

predicted_valid_lg = model_linereg.predict(features_valid_lg)
linereg_score = RMSE(target_valid_lg, predicted_valid_lg)

print('Среднее качество модели линейной регрессии:', linereg_score)

Среднее качество модели линейной регрессии: 2895.796221220404
CPU times: user 71.2 ms, sys: 65.8 ms, total: 137 ms
Wall time: 150 ms


### Случайный лес

Выбираем наилучшие параметры

In [ ]:
clf = RandomForestRegressor(random_state=1234)
parametrs = { 'n_estimators': range(18, 25, 3),
                     'max_depth': range(15, 25, 2),
                     'min_samples_leaf': range(2,5),
                     'min_samples_split': range(2,6,2) }
grid = GridSearchCV(clf, parametrs, cv=5, scoring=make_scorer(RMSE))
grid.fit(features_train, target_train)

print(grid.best_params_)
grid.best_score_

{'max_depth': 15, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 18}


1785.4667983913237

Запускаем модель обучения с лучшими параметрами.

In [ ]:
%%time
model_forest = RandomForestRegressor(random_state=1234, max_depth=15, min_samples_leaf=4, min_samples_split=2, n_estimators=18)
model_forest.fit(features_train, target_train)

CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 12.5 s


RandomForestRegressor(max_depth=15, min_samples_leaf=4, n_estimators=18,
                      random_state=1234)

In [ ]:
%%time
predicted_valid = model_forest.predict(features_valid)
forest_score = RMSE(target_valid, predicted_valid)
print('Качество модели Случайного леса:', forest_score)

Качество модели Случайного леса: 1769.7888705711002
CPU times: user 215 ms, sys: 4 ms, total: 219 ms
Wall time: 216 ms


### Градиентный бустинг LightGBM

Выбираем наилучшие параметры

In [ ]:
clf = lgb.LGBMRegressor()

parametrs = {'max_depth': range(-1, 10),
             'learning_rate': [0.001, 0.5],
             'n_estimators': [50, 100, 200, 500],
             }

grid = GridSearchCV(clf, parametrs, scoring=make_scorer(RMSE), cv=5)
grid.fit(features_train, target_train)

print(grid.best_params_)
grid.best_score_

{'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 50}


4443.824172203311

In [ ]:
%%time

model_lgb = lgb.LGBMRegressor(random_state=1234, max_depth=1, n_estimators=50, learning_rate=0.001)
model_lgb.fit(features_train, target_train)

CPU times: user 1.93 s, sys: 6.81 ms, total: 1.94 s
Wall time: 1.92 s


LGBMRegressor(learning_rate=0.001, max_depth=1, n_estimators=50,
              random_state=1234)

In [ ]:
%%time

predicted_valid = model_lgb.predict(features_valid)
lgb_score = RMSE(target_valid, predicted_valid)
print('Качество модели LightGBM:', lgb_score)

Качество модели LightGBM: 4444.609647052022
CPU times: user 133 ms, sys: 3 µs, total: 133 ms
Wall time: 94.8 ms


### Градиентный бустинг CatBoost

In [ ]:
clf = cb.CatBoostRegressor()

parametrs = {"iterations": [100],
                "depth": range(8, 16, 2),
                "learning_rate" : [0.15]
             }

grid = GridSearchCV(clf, parametrs, scoring=make_scorer(RMSE), cv=5)
grid.fit(features_train, target_train, verbose=10)

print(grid.best_params_)
grid.best_score_

0:	learn: 4087.7210605	total: 84.2ms	remaining: 8.34s
10:	learn: 2373.3218538	total: 381ms	remaining: 3.08s
20:	learn: 2082.3341099	total: 672ms	remaining: 2.53s
30:	learn: 1980.7083354	total: 954ms	remaining: 2.12s
40:	learn: 1922.5131386	total: 1.25s	remaining: 1.8s
50:	learn: 1884.0387586	total: 1.52s	remaining: 1.46s
60:	learn: 1858.9404781	total: 1.8s	remaining: 1.15s
70:	learn: 1834.0093821	total: 2.08s	remaining: 849ms
80:	learn: 1813.7062692	total: 2.36s	remaining: 553ms
90:	learn: 1796.8729778	total: 2.63s	remaining: 260ms
99:	learn: 1781.7518870	total: 2.88s	remaining: 0us
0:	learn: 4068.5506695	total: 30ms	remaining: 2.97s
10:	learn: 2360.8349496	total: 328ms	remaining: 2.65s
20:	learn: 2069.9068381	total: 622ms	remaining: 2.34s
30:	learn: 1973.3843276	total: 897ms	remaining: 2s
40:	learn: 1913.9699102	total: 1.18s	remaining: 1.7s
50:	learn: 1876.4317576	total: 1.46s	remaining: 1.4s
60:	learn: 1848.7101931	total: 1.75s	remaining: 1.12s
70:	learn: 1823.3655177	total: 2.03s	re

1818.4077144272487

In [ ]:
%%time
model_cb = cb.CatBoostRegressor(random_state=1234, max_depth=8, iterations=100, learning_rate=0.15)
model_cb.fit(features_train, target_train)


0:	learn: 4084.9823886	total: 38.8ms	remaining: 3.84s
1:	learn: 3723.6141605	total: 78.6ms	remaining: 3.85s
2:	learn: 3424.2377701	total: 116ms	remaining: 3.76s
3:	learn: 3185.1825498	total: 154ms	remaining: 3.71s
4:	learn: 2991.0752580	total: 191ms	remaining: 3.62s
5:	learn: 2833.1288370	total: 227ms	remaining: 3.55s
6:	learn: 2700.2442018	total: 265ms	remaining: 3.52s
7:	learn: 2592.5888296	total: 304ms	remaining: 3.5s
8:	learn: 2507.1797788	total: 341ms	remaining: 3.44s
9:	learn: 2434.7971975	total: 377ms	remaining: 3.4s
10:	learn: 2376.0848098	total: 413ms	remaining: 3.34s
11:	learn: 2325.3830322	total: 449ms	remaining: 3.29s
12:	learn: 2280.4131599	total: 488ms	remaining: 3.26s
13:	learn: 2239.9816132	total: 523ms	remaining: 3.21s
14:	learn: 2209.9809565	total: 556ms	remaining: 3.15s
15:	learn: 2183.3516983	total: 592ms	remaining: 3.11s
16:	learn: 2153.7227410	total: 630ms	remaining: 3.07s
17:	learn: 2133.6212907	total: 663ms	remaining: 3.02s
18:	learn: 2109.9697827	total: 702ms	r

In [ ]:
%%time
predicted_valid = model_cb.predict(features_valid)
cb_score = RMSE(target_valid, predicted_valid)
print('Качество модели CatBoost', cb_score)

Качество модели CatBoost 1806.9210297933698
CPU times: user 18.7 ms, sys: 42 µs, total: 18.7 ms
Wall time: 16.8 ms


## Анализ моделей

In [ ]:
columns = ['Модель', 'Время обучение модели, сек.', 'Время предсказания модели, сек.', 'RMSE',]
model_lineregression = ['Линейная регрессия', 12.2, 0.084, linereg_score]
model_forest = ['Случайный лес', 9.87, 0.275,  forest_score]
model_lgbm = ['LGBMRegressor', 2.71, 0.511, lgb_score]
model_cbust = ['CatBoostRegressor', 2.07, 0.014, cb_score]

table = pd.DataFrame([model_lineregression, model_forest, model_lgbm, model_cbust], columns = columns)
display(table)

,Модель,"Время обучение модели, сек.","Время предсказания модели, сек.",RMSE
0,Линейная регрессия,12.20,0.084,2895.796221
1,Случайный лес,9.87,0.275,1769.788871
2,LGBMRegressor,2.71,0.511,4444.609647
3,CatBoostRegressor,2.07,0.014,1806.921030


В результате по качеству обучения моделей на тренировочной выборке наилучие результаты показала модель LGBMRegressor. Проверим эту модель на тестовой выборке.

In [ ]:
%%time

model = lgb.LGBMRegressor(random_state=1234, max_depth=5, n_estimators=500, learning_rate=0.5)
model.fit(features_train, target_train)
predict = model.predict(features_test)
score = RMSE(target_test, predict)
print('Качество модели градиентного бустинга LightGBM на тестовой выборке:', score)

Качество модели градиентного бустинга LightGBM на тестовой выборке: 1667.0672923739592
CPU times: user 7min 55s, sys: 4.79 s, total: 8min
Wall time: 8min 3s


### Итоги.
Целью данной работы было построение модели предсказания рыночной стоимости автомобиля с пробегом на основе данных о технических характеристиках, комплектации и ценах других автомобилей. Важными критериями качества модели являются качество предсказания, время обучения модели и время предсказания модели.
<br/>
<br/>
В процессе работы с датасетом была проведена подготовка данных для будущего обучения модели. В результате предобработки данных были заполнены пропуски, исправлены аномальные значения, удалены неактульные для обучения столбцы. 
<br/>
<br/>
В процессе выбора наилучшей модели обучения на тренировочной выборке обучили 4 модели, замерили время их работы и качество предсказаний. В качестве моделей использованы Линейная регрессия, Случайный лес, LGBMRegressor, CatBoostRegressor.
<br/>
<br/>
Наилучшие результаты на показала модель LGBMRegressor c RMSE = 385.92 и временем обучения 72 секунды. На тестовой выборке модель также показала высокое качество. 
В дальнейшем рекомендуем применять модель LGBMRegressor.